# Word embeddings i Python

I denne notebook vises, hvordan man kan træne sin egen word embeddings model med `gensim` i Python.

In [ ]:
# indlæser pakker
import os
from os.path import join
import pandas as pd
from tqdm import tqdm # progress bar

# spacy
import spacy

# gensim
from gensim.models import Word2Vec # word2vec model
import gensim.downloader # download funktion til at hente eksisterende modeller/vectors med gensim


# indlæs sprogmodel
nlp = spacy.load('da_core_news_md')
nlp_en = spacy.load('en_core_web_md')

## Træn din egen model

Vi træner her en word embeddings model med referater for møder i Folketinget.

Først skal data indlæses og sættes i korrekt format - `gensim` forventer input i form af en *liste med lister af tokens* - en liste af tokens per tekst.

**Indlæs og sæt data i korrekt format**

In [ ]:
file_p = join('/work', '83232', 'data', 'dk_parl', 'dkparl_simple_20221216.json') # sti til data

In [ ]:
parldata = pd.read_json(file_p, orient = 'records') # indlæs med pandas

In [ ]:
parldata = parldata.explode('items') # omdan lister af tekster til rækker - en række per tekst

In [ ]:
parldata = parldata.dropna() # fjerner missing

In [ ]:
parldata['date'] = pd.to_datetime(parldata['DateOfSitting']) # konverter datoformat

In [ ]:
parldata.head()

In [ ]:
parldata.shape

In [ ]:
parldata2010 = parldata.loc[parldata['date'].dt.year == 2010, :] # udvælger data fra 2010

**Tokenize med `spaCy`**

In [ ]:
parltexts = parldata2010['items'].str.lower() # liste af tekster i lower-case

tokenized = [] # tom liste af pre-processed tekst

nlp_tokenize = spacy.load('da_core_news_md', enable=['tokenizer']) # indlæser sprogmodel på ny men bruger kun tokenizer

# looper igennem hver tekst og tokenizer - nlp.pipe behandler tekster i batches.
# tqdm er blot for at få en progress bar - skal bruge en "total" (tekster i alt) for at udregne resterende tid.

for doc in tqdm(nlp_tokenize.pipe(parltexts), total=len(parltexts)): 
    # liste til tokens i enkelt tekst
    text_tokens = []
    
    # loop igennem tokens i doc-objekt
    for token in doc:
        text_tokens.append(token.text) # tilføj token til token-liste

    # tilføj tekst tokens til samlede liste
    tokenized.append(text_tokens)

**Træn model med `gensim`**

In [ ]:
model = Word2Vec(sentences=tokenized,  # input data
                 vector_size=50,       # antal dimensioner (størrelse på embedding)
                 window=5,             # kontekstvindue (hvor mange ord skal tælles som del af kontekst?)
                 sg=1,                 # cbow eller skip-gram (cbow=0, sg=1)
                 negative=5,           # antal negative samples den skal danne for hvert tekststykke (jo flere, jo længere beregningstid)
                 min_count=3,          # minimumsgrænse for hvor mange gange ordet skal optræde
                 workers=8)            # antal CPU kerner (tjek din maskine)

**Mest lignende ord**

In [ ]:
model.wv.most_similar('danmark', topn=10)

In [ ]:
model.wv.most_similar('danskerne', topn=10)

In [ ]:
model.wv.most_similar('eu', topn=10)

**Sammenlign ord**

In [ ]:
model.wv.similarity('afghanistan', 'irak')